In [1]:
'''
Resnet for face recognition

MLAI Project

yidongh@andrew.cmu.edu

2019-10-31
'''
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset

cuda = torch.cuda.is_available()
cuda

True

## Resnet Definition

In [2]:
# Define the architecture
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=dilation,
                    groups=groups, bias = False, dilation=dilation)

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']
    
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                base_width=64, dilation=1, norm_layer = None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups!=1 or base_width!=64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
            
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True) # Notice this operation. BatchNorm doesn't need its output to do back-prop. Bui conv2d needs.
        self.conv2 = conv3x3(planes,planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out
    
class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']
    
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width/64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width,planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out


In [3]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 2300, zero_init_residual = False,
                groups = 1, width_per_group = 64, replace_stride_with_dilation=None,
                norm_layer = None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be none"
                            "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        
        # This could be modified. If stride = 2, then 32->16. Now I change it from 2 to 1.
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=1, stride=1, padding=0,
                              bias = False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace = True)
        # Maxpool might be unnecessary
        #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=1, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.b2.weight, 0)
                    
    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride!=1 or self.inplanes != planes*block.expansion:
            downsample = nn.Sequential(
            conv1x1(self.inplanes,planes*block.expansion,stride),
            norm_layer(planes*block.expansion))
            
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))
            
        return nn.Sequential(*layers)
    
    def _forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # x = self.maxpool(x)
        #print('1',x.shape)
        
        x = self.layer1(x)
        #print('2',x.shape)
        x = self.layer2(x)
        #print('3',x.shape)
        x = self.layer3(x)
        #print('4',x.shape)
        x = self.layer4(x)
        #print('5',x.shape)
        
        x = self.avgpool(x)
        #print('6',x.shape)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x
        
    forward = _forward
    
def _resnet(arch, block, layers, **kwargs):
    model = ResNet(block, layers, **kwargs)
    return model

def resnet18(**kwargs):
    return _resnet('resnet18', BasicBlock, [2,2,2,2],**kwargs)

def resnet34(**kwargs):
    return _resnet('resnet34', BasicBlock, [3,4,6,3],**kwargs)

def resnet38(**kwargs):
    return _resnet('resnet38', Bottleneck, [3,3,3,3],**kwargs)

def resnet50(**kwargs):
    return _resnet('resnet50', Bottleneck, [3,4,6,3],**kwargs)

def resnet101(**kwargs):
    return _resnet('resnet101', Bottleneck, [3,4,23,3],**kwargs)

def resnet152(**kwargs):
    return _resnet('resnet152', Bottleneck, [3,8,36,3],**kwargs)

## Data Loading

In [4]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        
        return img, label

In [5]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    uniqueID_list = []
    for root, directories, filenames in os.walk(datadir):
        b = -1
        for filename in filenames:
            if filename.endswith('.jpg'):
                if(filename[0]+filename[1]=='._'):
                    filename = filename[2:]
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                a = root.split('/')[-1]
                if a!=b:
                    uniqueID_list.append(a)
                b=a
                
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n)))
    reverse_dict = dict(zip(range(class_n),uniqueID_list))
    label_list = [target_dict[ID_key] for ID_key in ID_list]
    
    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n, reverse_dict

In [6]:
train_img_list, train_label_list, train_class_n, reverse_dict = parse_data('train_data/medium')
val_img_list,val_label_list, val_class_n, val_dict = parse_data('validation_classification/medium')
#test_img_list, test_label_list, test_class_n, _ = parse_data('test_c')

trainset = ImageDataset(train_img_list, train_label_list)
valset = ImageDataset(val_img_list, val_label_list)
#testset = ImageDataset(test_img_list, test_label_list)
#testset = torchvision.datasets.ImageFolder(root='test_c/test_classification/',transform=torchvision.transforms.ToTensor())

train_dataloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=3,drop_last=False)
val_dataloader = DataLoader(valset, batch_size=128, shuffle=False, num_workers=3)
#test_dataloader = DataLoader(testset[0], batch_size=1, shuffle=False,num_workers=3)


#Images		#Labels
822154		2300
#Images		#Labels
4601		2300


## CenterLoss + CrossEntropyLoss

In [7]:
class CenterLoss(nn.Module):
    """
    Args:
        num_classes (int): number of classes.
        feat_dim (int): feature dimension.
    """
    def __init__(self, num_classes, feat_dim, device = torch.device('cuda')):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.device = device
        self.centers = nn.Parameter(torch.randn(self.num_classes,self.feat_dim).to(self.device))
    
    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())
        
        classes = torch.arange(self.num_classes).long().to(self.device)
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = []
        for i in range(batch_size):
            value = distmat[i][mask[i]]
            value = value.clamp(min=1e-12, max=1e+12) # for numerical stability
            dist.append(value)
        dist = torch.cat(dist)
        loss = dist.mean()

        return loss

        

In [8]:
def train(model, data_loader, test_loader, task = 'Classification'):
    model.train()
    
    for epoch in range(numEpochs):
        avg_loss = 0
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer_label.zero_grad()
            optimizer_closs.zero_grad()
            
            feature, outputs = model(feats)
            
            l_loss = criterion_label(outputs, lables.long())
            c_loss = criterion_closs(feature, labels.long())
            
            loss = l_loss + closs_weight*c_loss
            
            loss.backward()
            optimizer_label.step()
            
            for param in criterion_closs.parameters():
                param.grad.data *= (1. / closs_weight)
            optimizer_closs.step()
            
            avg_loss += loss.item()
            if batch_num % 1000 == 999:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/1000))
                avg_loss = 0.0    
        
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
        if task == 'Classification':
            val_loss, val_acc = test_classify(model, test_loader)
            train_loss, train_acc = test_classify(model, data_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))
            


In [9]:
def train1(model, data_loader, test_loader, task = 'Classification'):
    model.train()
    
    for epoch in range(numEpochs):
        avg_loss = 0
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer_label.zero_grad()
            
            outputs = model(feats)
            
            l_loss = criterion_label(outputs, labels.long())
            
            loss = l_loss
            
            loss.backward()
            optimizer_label.step()
            
            avg_loss += loss.item()
            if batch_num % 1000 == 999:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/1000))
                avg_loss = 0.0    
        
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
        if task == 'Classification':
            val_loss, val_acc = test_classify(model, test_loader)
            train_loss, train_acc = test_classify(model, data_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))

In [10]:
def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats) ####
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        l_loss = criterion_label(outputs, labels.long())
        #c_loss = criterion_closs(feature, labels.long())
        loss = l_loss #+ closs_weight * c_loss
        #loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total

In [11]:
#train_test_image_list, train_test_label_list, n_test_class = parse_data('train_test')

## Train the network

In [13]:
numEpochs = 3

learningRate = 1e-2
weightDecay = 8e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
network = resnet38()
print(network)
#closs_weight = 1
lr_cent = 0.5
#criterion_closs = CenterLoss(num_classes = 2300, feat_dim = 64, device = device)
criterion_label = nn.CrossEntropyLoss()
optimizer_label = optim.Adam(network.parameters())
#optimizer_closs = torch.optim.SGD(criterion_closs.parameters(), lr=lr_cent)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [15]:
print(network.modules)

<bound method Module.modules of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, a

In [16]:
network.to(device)
network.train()
for i in range(10):
    print("this is "+str(i)+"th iteration")
    train1(network, train_dataloader, val_dataloader)
    #torch.save(network.state_dict(), './model10_epoch'+str((i+1)*2)+'.pth')

this is 0th iteration


RuntimeError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 10.91 GiB total capacity; 6.34 GiB already allocated; 25.06 MiB free; 124.26 MiB cached)

In [16]:
torch.save(network.state_dict(),'Resnet38_iteration1_epoch2_dropout.pth')

## Test

In [17]:
network.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): 

In [5]:
test_img_list  = []
for root, dictionaries, filenames in os.walk('test_classification/medium/'):
    for filename in filenames:
        img_path = os.path.join(root, filename)
        test_img_list.append(img_path)
        
test_img_list.sort()

In [6]:
print(test_img_list[0:10])

['test_classification/medium/5000.jpg', 'test_classification/medium/5001.jpg', 'test_classification/medium/5002.jpg', 'test_classification/medium/5003.jpg', 'test_classification/medium/5004.jpg', 'test_classification/medium/5005.jpg', 'test_classification/medium/5006.jpg', 'test_classification/medium/5007.jpg', 'test_classification/medium/5008.jpg', 'test_classification/medium/5009.jpg']


In [20]:
class testDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        
        return img

In [21]:
testset = testDataset(test_img_list)
test_dataLoader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=3)

In [22]:
pred_list = []
for batch_num, (feats) in enumerate(test_dataLoader):
    feats = feats.to(device)
    prediction = network(feats)
    _, pred_labels = torch.max(F.softmax(prediction, dim=1), 1)
    pred_labels = pred_labels.view(-1)
    pred_list.append(pred_labels.cpu().numpy())

In [23]:
print(pred_list[0].dtype)

int64


In [24]:
pred_list = np.array(pred_list)
real_list = np.zeros_like(pred_list)
for i in range(pred_list.shape[0]):
    real_list[i] = reverse_dict[int(pred_list[i])]


In [12]:
print(reverse_dict)

{0: '857', 1: '2071', 2: '575', 3: '819', 4: '2254', 5: '629', 6: '2136', 7: '1001', 8: '2230', 9: '480', 10: '1084', 11: '1303', 12: '1588', 13: '342', 14: '1884', 15: '1882', 16: '1408', 17: '745', 18: '1157', 19: '464', 20: '669', 21: '1393', 22: '1749', 23: '1271', 24: '2161', 25: '1576', 26: '1283', 27: '1493', 28: '708', 29: '1189', 30: '1196', 31: '2234', 32: '1535', 33: '499', 34: '642', 35: '2214', 36: '865', 37: '1459', 38: '422', 39: '696', 40: '245', 41: '474', 42: '1251', 43: '1496', 44: '1295', 45: '74', 46: '578', 47: '1626', 48: '2133', 49: '98', 50: '1128', 51: '500', 52: '513', 53: '110', 54: '1429', 55: '2073', 56: '1959', 57: '25', 58: '963', 59: '1262', 60: '954', 61: '813', 62: '1212', 63: '1207', 64: '2209', 65: '1777', 66: '1979', 67: '1044', 68: '782', 69: '469', 70: '128', 71: '1668', 72: '320', 73: '1936', 74: '1642', 75: '1059', 76: '12', 77: '170', 78: '326', 79: '767', 80: '2227', 81: '2027', 82: '951', 83: '1123', 84: '1093', 85: '1422', 86: '2141', 87: '

In [25]:
print(real_list[0:10])

[[ 947]
 [ 317]
 [1277]
 [2115]
 [ 863]
 [ 384]
 [1484]
 [ 172]
 [ 702]
 [ 245]]


In [26]:
np.savetxt("resnet34_iteration1.csv",real_list,delimiter=",")

In [27]:
print(reverse_dict.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [28]:
print(reverse_dict.values())

dict_values(['357', '1961', '1943', '387', '1320', '183', '1530', '1862', '1207', '1820', '296', '1935', '1126', '994', '971', '1274', '1023', '3', '1924', '2038', '433', '692', '2086', '1346', '2043', '70', '2030', '420', '1596', '1053', '236', '1103', '323', '396', '518', '438', '329', '1567', '1624', '201', '766', '1946', '890', '1478', '1069', '1605', '2157', '970', '909', '801', '268', '1834', '1726', '870', '881', '39', '318', '1172', '515', '278', '2037', '921', '910', '1363', '350', '2283', '777', '1398', '832', '1562', '1224', '1557', '1680', '104', '1026', '294', '513', '2294', '2218', '102', '2203', '740', '1142', '1220', '209', '635', '526', '1837', '121', '1713', '432', '739', '764', '495', '1760', '861', '1302', '2050', '1221', '1213', '1440', '858', '763', '544', '312', '1148', '2065', '1013', '1183', '1452', '1008', '1981', '914', '1888', '113', '779', '1604', '2019', '1656', '1558', '2117', '1949', '860', '1780', '446', '637', '412', '347', '123', '556', '439', '1695',

In [17]:
torch.cuda.memory_allocated(device)

6808130048